# Text

## TODO
- [ ] how to convert dataset ?
- [ ] what' needed ? text ?

&nbsp;  
## Notes
- leave out emotional and code switched
- finetune (feedback loop) VS pretrain (unsupervised, clustering data)
- 7,000h of unlabeled data (=> use Granary pipeline tweaked ?)
- GPT2 knows arabic already for XTTS, training it would be good and making the Maltese char understood or phonems
- how to make it expressive? See Rasa (the minimum amount of expressive data needed per emotion (ii) the difficulty of synthesizing certain emotions, and (iii) the better expressivity of multi-emotion systems over single-emotion systems.) :
`tts.tts_to_file(text="This is a test.", file_path=OUTPUT_PATH, emotion="Happy", speed=1.5)`  
- Use Adam optimizer

&nbsp;  
## References
- <https://huggingface.co/coqui/XTTS-v2>  
- <https://docs.coqui.ai/en/latest/models/xtts.html#training>  
- <https://huggingface.co/spaces/coqui/xtts>  
- <https://github.com/anhnh2002/XTTSv2-Finetuning-for-New-Languages%3E>
- <https://github.com/coqui-ai/TTS/blob/dev/recipes/ljspeech/xtts_v2/train_gpt_xtts.py>
- <https://docs.coqui.ai/en/latest/models/xtts.html#advanced-training>
- <https://colab.research.google.com/drive/1GiI4_X724M8q2W-zZ-jXo7cWTV7RfaH-?usp=sharing#scrollTo=Th91ofnQWr8Y>
- <https://github.com/daswer123/xtts-webui?tab=readme-ov-file> + <https://colab.research.google.com/drive/1MrzAYgANm6u79rCCQQqBSoelYGiJ1qYL#scrollTo=4arC1ywzqfcu>
- <https://github.com/erew123/alltalk_tts/wiki/XTTS-Model-Finetuning-Guide-(Simple-Version)>

## Knowledge

Core Concepts
Understanding Finetuning
Finetuning is the process of adjusting a pre-trained model using new, specific voice data to better capture unique vocal characteristics. This is useful for creating personalized voice models that retain the base model's general abilities while improving on the nuances of your specific dataset.

When to Finetune:

Base model doesn’t capture desired vocal qualities.
Adaptation needed for accents, unique voices, or specific speaking styles.
Custom voices for particular applications.
Preparing Audio Data
Duration: Minimum of 3 minutes, 5–10 minutes recommended.
Content: Clear, noise-free, and consistent volume. Avoid background sounds or music.
Format: MP3, WAV, or FLAC. Any sample rate works, and stereo or mono is supported.
Tips:

Use varied sentence lengths, tones, and speaking speeds.
Pre-process to remove background noise, split long files if needed, and ensure content quality.
Training Process Overview
Dataset Generation:

Audio is segmented and transcribed using Whisper.
Dataset split into training and evaluation sets (metadata_train.csv and metadata_eval.csv).
Use the Dataset Validation tab to look for inconsistencies in the transcriptions & correct as necessary.
Model Training:

Set training parameters and monitor progress. Model learns to emulate the target voice.
Validation Paths: Paths for validation data and Whisper model transcription are used to validate and monitor quality. Customize these paths if you want to specify a different dataset for validation.
Testing:

Run tests with different text inputs to evaluate quality, pronunciation, and emotional range.
Model Export:

Compact model, save essential files, and clean up training artifacts.
After the model is compacted and moved, select the wav files from the models folder for using with voice cloning Details here.
3. Technical Deep Dives
Training Parameters
Epochs
An epoch is a full pass through the dataset. Choose the number of epochs based on the desired outcome:

Standard voices: 10–20 epochs
Highly unique voices or accents: 40+ epochs
Complex voices or new languages: May require 100+ epochs.
Tip: Monitor loss values to avoid overfitting. See Overtraining.

Batch Size
Small Batch Size (4–8): Lower memory use, more frequent updates.
Large Batch Size (32–64): Faster processing, requires more memory.
Gradient Accumulation: If VRAM is limited, simulate larger batch sizes with gradient accumulation:

batch_size = 8
gradient_accumulation_steps = 4  # Effective batch size = 32
Learning Rate
Controls how quickly the model learns:

1e-6 to 5e-6: Stable, slow learning.
1e-5: Balanced, suitable for most cases.
1e-3 and higher: Fast but can be unstable.
Schedulers: Use learning rate scheduling (e.g., cosine annealing or exponential decay) for better results over long training runs.

Memory Management
Windows: Can use system RAM as extended VRAM.
Linux: Limited to physical VRAM, requiring higher VRAM for reliable operation.
Optimization Strategies:

Lower batch size or increase gradient accumulation if memory is limited.
Adjust worker threads and limit audio length for efficiency.
Signal Handling: Use the GUI’s stop option or standard interrupts to safely halt training without corrupting model state.
Advanced Features
BPE Tokenization
The script uses Byte-Pair Encoding (BPE) tokenization, which helps the model handle complex text, diverse languages, accents, and dialects more effectively. This feature allows the model to better manage unique vocabularies and speech patterns.

4. Troubleshooting & Optimization
Improving Model Quality
You can read advanced sections on this topic here

Signs of Good Training Progression:

Loss values consistently decrease over epochs. Example:
Text Loss: 0.02 -> 0.015 -> 0.012
Mel Loss: 4.5 -> 3.8 -> 3.2
Average Loss: 4.0 -> 3.5 -> 3.0
Model saves as "BEST MODEL" at new performance milestones.
Recognizing Overtraining:

Loss values plateau or start increasing, e.g.,:
Epoch 5: Text Loss = 0.009, Mel Loss = 2.9
Epoch 7: Text Loss = 0.010, Mel Loss = 3.1  # Performance worsening
Solutions: Implement early stopping, reduce training epochs, or lower learning rate.
Performance Optimization
Hardware:

Adjust batch size based on GPU capability.
Optimize worker count and monitor VRAM.
Configuration:

Use efficient settings, e.g., float16 precision if supported.
Regularly monitor memory and processing efficiency.
Adjust gradient_accumulation_steps for limited VRAM.
5. Deploying Your Model
Model Export and Storage
After finetuning, export and organize the model files:

Essential Files:

model.pth: Main model
config.json: Configuration settings
vocab.json: Tokenizer vocabulary
speakers_xtts.pth: Speaker embeddings
dvae.pth: Discrete Variational Autoencoder (DVAE) model file
mel_stats.pth: Mel spectrogram statistics





1000 EPOCH FOR TRAINING (https://github.com/erew123/alltalk_tts?#-finetuning-a-model)



---


# Inference on base model

In [ ]:
!pip install TTS
!pip install transformers==4.38.2 # to fix AttributeError: 'GPT2InferenceModel' object has no attribute 'generate'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 61.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 48.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 76.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 35.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 118.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.2
    Uninstalling transformers-4.53.2:
      Successfully uninstalled transformers-4.53.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.38.2 which is incompatible.


### Test on arabic

In [ ]:
import torch
from TTS.api import TTS
import time

# Fix for global not allowed
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import XttsAudioConfig, XttsArgs
from TTS.config.shared_configs import BaseDatasetConfig
import torch.serialization
torch.serialization.add_safe_globals([XttsConfig, XttsAudioConfig, BaseDatasetConfig, XttsArgs])

os.environ["COQUI_TOS_AGREED"] = "1"

reference_speaker_wav = "audio_arabic_speaker.mp3" #3-10s, high quality, works best with wav (https://huggingface.co/datasets/mozilla-foundation/common_voice_17_0/viewer/ar)

arabic_text = "تعتبر اللغة العربية من أغنى اللغات في العالم من حيث المفردات والتراكيب، وتتميز بجمال خطها وتاريخها العريق."
inference_language = "ar"
output_wav_path = f"output_arabic_zero_shot.wav"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

print("Loading Coqui XTTS-v2 model... (This may take a moment on first run)")
try:
  tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", progress_bar=True).to(device)

  print(f"Synthesizing Arabic text: '{arabic_text} with {reference_speaker_wav} speaker'")
  try:
    initial_time = time.time()
    tts.tts_to_file(
      text=arabic_text,
      speaker_wav=reference_speaker_wav,
      language=inference_language,
      file_path=output_wav_path,
      emotion="Happy",
      speed=1.5
    )
    final_time = time.time()
    print(f"Audio saved to {output_wav_path}.")
    print(f"Inference time: {final_time - initial_time} seconds")
  except Exception as e:
    raise Exception(f"An error occurred during speech synthesis: {e}")
except Exception as e:
  raise Exception(f"Failed to load the TTS model. Please check your installation and internet connection. Error: {e}")

Using cuda
Loading Coqui XTTS-v2 model... (This may take a moment on first run)
 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 | | > y
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2


100%|█████████▉| 1.86G/1.87G [00:40<00:00, 31.0MiB/s]
100%|██████████| 1.87G/1.87G [00:40<00:00, 46.0MiB/s]
4.37kiB [00:00, 15.9kiB/s]

361kiB [00:00, 1.41MiB/s]
100%|██████████| 32.0/32.0 [00:00<00:00, 119iB/s]
 96%|█████████▌| 7.46M/7.75M [00:00<00:00, 25.1MiB/s]

 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
 > Using model: xtts


100%|██████████| 7.75M/7.75M [00:18<00:00, 25.1MiB/s]

Synthesizing Arabic text: 'تعتبر اللغة العربية من أغنى اللغات في العالم من حيث المفردات والتراكيب، وتتميز بجمال خطها وتاريخها العريق. with audio_arabic_speaker.mp3 speaker'
 > Text splitted to sentences.
['تعتبر اللغة العربية من أغنى اللغات في العالم من حيث المفردات والتراكيب، وتتميز بجمال خطها وتاريخها العريق.']
 > Processing time: 9.24390196800232
 > Real-time factor: 0.8885577456687728
Audio saved to output_arabic_zero_shot.wav.
Inference time: 9.276689529418945 seconds


### Full example on Maltese

In [ ]:
import torch
from TTS.api import TTS
import time

# Fix for global not allowed
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import XttsAudioConfig, XttsArgs
from TTS.config.shared_configs import BaseDatasetConfig
import torch.serialization
torch.serialization.add_safe_globals([XttsConfig, XttsAudioConfig, BaseDatasetConfig, XttsArgs])

 #3-10s, high quality, works best with wav (https://huggingface.co/datasets/mozilla-foundation/common_voice_17_0/viewer/mt)
reference_speaker_wav = "test_female_maltese_8s.mp3"

texts = ["Merħba f'Malta! It-temp illum huwa sabiħ ħafna.", "Bonġu, din hija prova tal-mudell XTTS bil-lingwa Maltija.", "Il-Karozza Porsche Tal-2022 G ̄handha Speed Fenomenali!"]
maltese_text = texts[1]
# Use a supported language with phonetic similarities (it, ar)
inference_language = "ar"
output_wav_path = f"output_maltese_zero_shot_{inference_language}.wav"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device}")

print("Loading Coqui XTTS-v2 model... (This may take a moment on first run)")
try:
  tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", progress_bar=True).to(device)

  print(f"Synthesizing Maltese text: '{maltese_text} with {reference_speaker_wav} speaker'")
  try:
    initial_time = time.time()
    tts.tts_to_file(
        text=maltese_text,
        speaker_wav=[reference_speaker_wav],
        language=inference_language,
        file_path=output_wav_path,
        emotion="Happy",
        speed=1.0,
        split_sentences=True,
        # speaker="Ana Florence",
    )
    final_time = time.time()
    print(f"Audio saved to {output_wav_path}.")
    print(f"Inference time: {final_time - initial_time} seconds")
  except Exception as e:
    raise Exception(f"An error occurred during speech synthesis: {e}")
except Exception as e:
  raise Exception(f"Failed to load the TTS model. Please check your installation and internet connection. Error: {e}")



# Using the model directly:

# from TTS.tts.configs.xtts_config import XttsConfig
# from TTS.tts.models.xtts import Xtts

# config = XttsConfig()
# config.load_json("/path/to/xtts/config.json")
# model = Xtts.init_from_config(config)
# model.load_checkpoint(config, checkpoint_dir="/path/to/xtts/", eval=True)
# model.cuda()

# outputs = model.synthesize(
#     "It took me quite a long time to develop a voice and now that I have it I am not going to be silent.",
#     config,
#     speaker_wav="/data/TTS-public/_refclips/3.wav",
#     gpt_cond_len=3,
#     language="en",
# )



---


# Fine-Tuning


## Configuration

### Example of config
396 samples on V202, 44 epochs, batch size of 7, grads 10, with a max sample length of 11 seconds.

I think of training a voice model as like a big dart board right, the epochs are the general area its going to land, the grads are further fine tuning it within that small area defined by the epochs over time, the maximum length is just the length of audio it will try to create audio for. The ones where I use 11 seconds vs 12 or 14 dont seem to be very different.

Or 11 epochs, 6 sample, 6 grads, 11 seconds and it was fine. I had 89 samples.

The real key are the samples. Whisper tends to scan a file for audio it can chunk but if it fails to recognize parts of it enough times it will discard the rest of the audio.

How to get around this? Load the main samples into audacity, mix down to mono and start highlighting sections of 1 sentence maximum, and then just press CTRL D to duplicate it, go through the whole thing, cut out any breathing by turning it into dead sound, to do that you highlight the breath and press CTRL L. dont delete it or youll fuck your vocal pacing.

Once youre done delete the one you were creating dupes from, go export audio, multiple files (make sure theyre unmuted or they wont export), then tick truncate audio before clip beginning and select a folder.

My audio format is WAV signed 16-bit PCM, MONO, 44100. I use 44.1 because whisper will reduce it down to 22050 if it wants to, it sounds better somehow using 44.1.

Using this method for making samples I went from whisper making a dataset of about 50 to 396.

## Gemini first version

In [ ]:
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
from TTS.utils.manage import ModelManager
from TTS.tts.datasets import load_tts_samples
from trainer import Trainer, TrainerArgs

config = XttsConfig()

model_name = "tts_models/multilingual/multi-dataset/xtts_v2"
config.load_json(f"/root/.local/share/tts/{model_name}/config.json")
ModelManager().download_model(model_name)

model = Xtts.init_from_config(config)
model.load_checkpoint(config, checkpoint_dir=f"/root/.local/share/tts/{model_name}/")


# This assumes your dataset is in the LJSpeech format with a 'metadata.csv' file
dataset_config = {
    "name": "your_maltese_dataset",
    "path": "path/to/your/dataset/",  # Directory containing wavs and metadata.csv
    "meta_file_train": "metadata.csv",
    "language": "mt",
}

ds = load_dataset("espnet/yodas-granary", "Maltese")

# The dataset should be a list of dictionaries, each with 'text' and 'audio_file' keys
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=256,
    eval_split_size=0.1,
)


training_args = TrainerArgs(
  output_path="xtts_maltese_finetuned",
  run_name="xtts_maltese_finetune",
  num_loader_workers=4,
  batch_size=32,
  epochs=100,  # Adjust based on your dataset size and desired training time
  lr=1e-5,
  save_step=1000,
  log_step=100,
  use_ddp=True,  # Set to False if not using distributed training
)


trainer = Trainer(
  training_args,
  config,
  output_path="xtts_maltese_finetuned",
  model=model,
  train_samples=train_samples,
  eval_samples=eval_samples,
)

# The fine-tuned model will be saved in the 'output_path' directory
trainer.fit()

## Gemini 2nd version

### Explanation


XTTS-v2 supports fine-tuning, which is the process of adapting a pre-trained model to a new language or specific voice data. While XTTS-v2 is a massively multilingual model, Maltese is considered a low-resource language, meaning it might not have been extensively represented in the original training data. Therefore, fine-tuning with a dedicated Maltese dataset is the recommended approach to achieve high-quality results.

&nbsp;  
1. Data Preparation:
  1. Collect Maltese Speech Data: You will need a dataset of Maltese audio recordings paired with their corresponding text transcripts.
  The (Mozilla Common Voice, MASRI-HEADSET, for text data, a "free-running text corpus for Maltese" from newspapers, websites, and official documents was developed in 2014)
  2. Format Your Dataset: The data needs to be organized in a specific way, typically an LJSpeech-style format, which includes a metadata.csv file and a wavs directory containing your audio recordings.
  3. Audio Requirements: Ensure your audio files are mono, with a sample rate of 22050 Hz, and encoded as Signed 16-bit PCM.
  4. Quality Control: High-quality, clean, noise-free audio with consistent volume is crucial. Avoid background sounds, music, or excessive clipping. Discard any audio that is significantly lower quality than the rest.
  5. Data Quantity: While XTTS-v2 can clone voices with minimal data (e.g., 6 seconds for inference), for fine-tuning a new language, a minimum of 3 minutes of audio is recommended, with 5-10 minutes being ideal. Some users have achieved good results with over 100 hours of audio.
  6. Diversity: Include varied sentence lengths, tones, and speaking speeds in your dataset to improve the model's expressive range.

2. Vocabulary Extension and Configuration:  
  For a new language like Maltese, you will likely need to extend the tokenizer's vocabulary to properly handle Maltese phonemes and unique speech patterns. XTTS models use a specialized Byte-Pair Encoding (BPE) tokenizer.  
  You'll need to adjust certain fields in the model's configuration file (config.json) to reflect your dataset's characteristics, such as the dataset path, run name, output path, and audio parameters (e.g., sampling rate).


3. Finetuning Process:
  - Adjust training parameters like the number of epochs, batch size, and learning rate. A smaller learning rate is generally recommended for fine-tuning to prevent the model from losing the features it learned during pre-training.
  - Monitor the loss values during training to track progress.
  - The XTTS codebase provides scripts and guides for fine-tuning.
Training Specific Parts of the Model
  - DVAE (VQ-VAE, Responsible for converting audio waveforms into discrete audio tokens) and GPT (GPT-2 Encoder, Predicts VQ-VAE audio codes based on text tokens) fine-tuning are often sufficient for achieving optimal results.  
  Fine-tuning the HiFiGAN decoder (Transforms latent vectors from the GPT-2 encoder into the final audio signal) has sometimes resulted in worse performance in user experiments.  
  For small datasets (e.g., around 10 hours), it's even recommended not to fine-tune the DVAE component.

4. (Optional) Improvements:
  - Prompt-Based Control Modules: XTTS already uses speaker prompt tokens to condition its GPT-based language model. You could integrate a module that allows for more explicit, natural language-based style and emotion prompts. This would enable users to specify desired expressive qualities (e.g., "speak sadly," "use a high pitch") via text, moving beyond reliance on reference audio for emotion transfer and potentially improving prosodic control.
  - Diffusion Models for Prosody: To address XTTS-v2's limitations in dynamic F0 variation and overall naturalness, a diffusion-based module could be introduced. This module could refine the mel-spectrograms or audio tokens generated by the GPT-2 encoder, or even replace parts of the acoustic modeling. Diffusion models are excellent at capturing complex prosodic variations and offer fine-grained control.
  - Latency Reduction Components: For real-time applications, you could implement architectural optimizations to reduce latency. This might involve modifying XTTS's input processing to handle incremental text inputs from Large Language Models (LLMs) more effectively, allowing speech generation to begin without waiting for the entire text response. Exploring single-stream speech codecs could also streamline internal audio representation for improved efficiency.

### Requirements

In [ ]:
!git clone https://github.com/coqui-ai/TTS.git
!cd TTS
!pip install -e .[all,dev]

### Steps

#### 1. Dataset formation
Organize your Maltese audio-text pairs. A common format is a metadata.csv (or .tsv) file with audio_filename|text lines.

In [ ]:
# Conceptual code for loading and preprocessing Common Voice Maltese
from datasets import load_dataset
import os

# Load Maltese Common Voice dataset
# Ensure you have accepted the terms on Hugging Face Hub
common_voice_mt = load_dataset("mozilla-foundation/common_voice_15_0", "mt", split="train", cache_dir="./common_voice_cache")

# Example of basic preprocessing (adapt as needed for your data)
# You'll need to handle audio path, ensure correct sampling rate (24kHz for XTTS)
# and clean text (normalize numbers, expand abbreviations, handle punctuation specific to Maltese)

output_dir = "maltese_tts_data"
os.makedirs(output_dir, exist_ok=True)
with open(os.path.join(output_dir, "metadata.csv"), "w", encoding="utf-8") as f:
    for item in common_voice_mt:
        # Assuming 'path' is the audio file path and 'sentence' is the transcription
        # You might need to move/copy audio files to output_dir/audios/
        audio_path = item['path'] # This might be a relative path or a URL, depends on dataset structure
        text = item['sentence']
        # Perform text normalization specific to Maltese here
        # Example: text = normalize_maltese_text(text)
        f.write(f"{audio_path}|{text}\n")
print(f"Prepared metadata.csv in {output_dir}")

#### Tokenizer extension

In [ ]:
# Conceptual code for creating/extending a tokenizer (using SentencePiece or Hugging Face tokenizers)
# This often involves training a new tokenizer on your Maltese text corpus
# or updating the existing one.
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

# Assuming you have a large text corpus of Maltese sentences
paths = [os.path.join(output_dir, "metadata.csv")] # Or a list of text files

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Train the tokenizer
tokenizer.train(files=paths, vocab_size=50000, min_frequency=2, special_tokens=[
    "<s>", "<pad>", "</s>", "<unk>", "<mask>",
])

# Save files
tokenizer_dir = "maltese_tokenizer"
os.makedirs(tokenizer_dir, exist_ok=True)
tokenizer.save_model(tokenizer_dir)

# Note: Integrating this new tokenizer with XTTS requires modifying model configuration.
# Coqui TTS config files (config.json) specify the tokenizer path.
# You would typically modify the tokenizer_path and vocab_size in your XTTS model config.

#### Model configuration
Copy an existing config.json for XTTS from the Coqui TTS repository (TTS/config_files/ljspeech-xtts-v2.json or similar) and modify it for Maltese:

model_args.speaker_encoder_config_path: Path to speaker encoder.

model_args.text_tokenizer_config_path: Point to your new Maltese tokenizer files.

characters: Update the character set if Maltese has unique characters not covered.

phonemes: If using phoneme input, define Maltese phoneme set. XTTS typically uses character-level.

num_speakers: If adding specific speakers, adjust. For zero-shot, this is often handled by embeddings.



#### Fine-Tuning script

In [ ]:
# Conceptual command for fine-tuning XTTS
# This is a highly simplified representation.
# Refer to Coqui TTS fine-tuning guides for exact parameters.
# Example: https://github.com/coqui-ai/TTS/blob/dev/docs/fine_tune_xtts.md

!python TTS/bin/train.py \
    --config_path /path/to/your/maltese_xtts_config.json \
    --coq_args.restore_path /path/to/pre_trained_xtts_model.pth \
    --coq_args.output_path /path/to/fine_tuned_model_output \
    --coq_args.datasets.0.name "your_maltese_dataset" \
    --coq_args.datasets.0.path /path/to/your/maltese_tts_data \
    --coq_args.datasets.0.meta_file_train "metadata.csv" \
    --coq_args.epochs 100 # Adjust epochs, learning rate, batch size etc.

VQ-VAE Fine-tuning (DVAE): Adapting the discrete autoencoder, especially if your target language has significantly different acoustic properties.

GPT-2 Encoder Fine-tuning: This is typically the main stage for language adaptation, learning the mapping from Maltese text to audio tokens.

End-to-end or Vocoder Fine-tuning (Optional): Fine-tuning the HiFi-GAN vocoder if needed, though XTTS's vocoder is often robust

#### Improvements

In [ ]:
# Conceptual PyTorch-style pseudo-code for training specific components
import torch
from TTS.api import TTS
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
from TTS.utils.audio import AudioProcessor

# 1. Load XTTS model and config
config_path = "/path/to/your/maltese_xtts_config.json"
model_checkpoint_path = "/path/to/pre_trained_xtts_model.pth" # Or your partially fine-tuned model

config = XttsConfig()
config.load_json(config_path)
model = Xtts(config)
model.load_checkpoint(config, checkpoint_path=model_checkpoint_path, eval=False)
model.train()

# 2. Define optimizer and loss function (standard for your task)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5) # Adjust learning rate
# criterion = YourLossFunction() # e.g., for reconstruction, speaker similarity etc.

# 3. Freeze/Unfreeze specific components
# Example: Only train the GPT-2 Encoder (acoustic model)
for name, param in model.named_parameters():
    if "gpt" in name: # Assuming your GPT-2 encoder parameters contain "gpt"
        param.requires_grad = True
    else:
        param.requires_grad = False

# Example: Only train the VQ-VAE (if adapting for new acoustic properties)
# This might require a separate VQ-VAE training pipeline as in Coqui TTS
# For in-place fine-tuning, you might access specific sub-modules:
# for name, param in model.model.dvae.named_parameters(): # Assuming 'dvae' is the VQ-VAE module
#     param.requires_grad = True

# 4. Training Loop (conceptual)
# Iterate through your Maltese dataset
# For each batch:
#   inputs = preprocess_text_and_audio_batch(maltese_data_batch)
#   outputs = model(inputs) # Forward pass
#   loss = criterion(outputs, targets)
#   loss.backward()
#   optimizer.step()
#   optimizer.zero_grad()

##### **Prompt-Based Expressive Control**:
Idea: Inject additional conditioning signals derived from natural language prompts ("speak sadly", "use a fast pace") into the GPT-2 Encoder.

&nbsp;  
Integration:
- Style Encoder: You could train a separate "prompt encoder" or "style text encoder" that takes a natural language prompt and converts it into an embedding. This embedding would then be concatenated with the existing speaker latents or fed into the GPT-2 Encoder's attention mechanism as an additional conditioning vector.
- Modifying GPT-2 Input: Adjust the input sequence to the GPT-2 encoder to include these new "style tokens" along with text tokens and speaker latents. This would likely require changes to the GPT-2 model's input embedding layer and attention mechanisms.

&nbsp;  
Python (Highly abstract - would involve modifying Xtts class methods):
```python
class XttsWithPromptControl(Xtts):
    def __init__(self, config):
        super().__init__(config)
        self.prompt_encoder = YourPromptEncoderModel() # New module for prompts

    def forward(self, text_input, speaker_ref_audio, prompt_text):
        # Existing XTTS logic for text and speaker
        speaker_latents = self.model.speaker_encoder(speaker_ref_audio)
        text_tokens = self.tokenizer.encode(text_input)

        # New: Encode prompt text
        prompt_embedding = self.prompt_encoder(prompt_text)

        # Combine speaker_latents, text_tokens, and prompt_embedding
        # This is the tricky part: how to effectively inject prompt_embedding
        # into the GPT-2 encoder's processing.
        # Could be: concatenation, cross-attention layer, or modifying existing conditioning.

        # modified_conditioning = torch.cat([speaker_latents, prompt_embedding], dim=-1)
        # audio_tokens = self.model.gpt(text_tokens, modified_conditioning)

        # Then pass to vocoder
        # audio_waveform = self.model.vocoder(audio_tokens)
        # return audio_waveform
        pass # Placeholder

```

##### **Diffusion Models for Prosody (Post-Acoustic Model Refinement)**
Idea: Use a diffusion model to refine the prosody of the acoustic features (mel-spectrograms or VQ-VAE tokens) generated by XTTS's GPT-2 Encoder.

&nbsp;  
Integration:
- Post-processing Module: Train a separate diffusion-based prosody model that takes the output of XTTS's GPT-2 Encoder (e.g., predicted VQ-VAE codes or intermediate mel-spectrograms) and refines their prosodic contours (pitch, duration, energy) before sending them to the HiFi-GAN vocoder.

- Training: This would be a two-stage training process: first XTTS, then the diffusion model trained on XTTS's output and natural speech to learn prosodic transformations.

&nbsp;  
Python (Highly abstract - assumes you've got XTTS output):
```python
class ProsodyRefinerDiffusion(torch.nn.Module):
  def __init__(self):
    super().__init__()
    # Define your diffusion model architecture (e.g., U-Net for mel-spectrograms)
    # with conditioning on text and speaker embeddings

  def forward(self, xtts_acoustic_output, text_embeddings, speaker_embeddings):
    # Apply diffusion process to refine prosody of xtts_acoustic_output
    # conditioned on text and speaker.
    refined_acoustic_output = self.diffusion_process(xtts_acoustic_output, text_embeddings, speaker_embeddings)
    return refined_acoustic_output

# In your main XTTS inference pipeline:
# ... after xtts_acoustic_output from model.gpt ...
# prosody_refiner = ProsodyRefinerDiffusion()
# refined_output = prosody_refiner(xtts_acoustic_output, text_embeddings, speaker_latents)
# audio_waveform = model.model.vocoder(refined_output)
```

## Fine-Tuning tuto
<https://github.com/anhnh2002/XTTSv2-Finetuning-for-New-Languages>

## [Advanced training](https://docs.coqui.ai/en/latest/models/xtts.html#advanced-training)

From : <https://github.com/coqui-ai/TTS/blob/dev/recipes/ljspeech/xtts_v2/train_gpt_xtts.py>

It uses a GPTTrainerConfig which includes optimizer settings. It uses AdamW, which is a common and effective variant of Adam, with specific parameters (betas, eps, weight_decay).
optimizer="AdamW",
optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
lr=5e-06, # learning rate

In [ ]:
import os

from trainer import Trainer, TrainerArgs

from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig, XttsAudioConfig
from TTS.utils.manage import ModelManager


# Logging parameters
RUN_NAME = "GPT_XTTS_v2.0_Maltese_FT" # Changed run name
PROJECT_NAME = "XTTS_Maltese_trainer" # Changed project name
DASHBOARD_LOGGER = "tensorboard"
LOGGER_URI = None

# Set here the path that the checkpoints will be saved. Default: ./run/training/
# IMPORTANT: Change this to your desired output directory for the fine-tuned model
OUT_PATH = "xtts_maltese_finetuned" # Example output path, change as needed

# Training Parameters
OPTIMIZER_WD_ONLY_ON_WEIGHTS = True
START_WITH_EVAL = True
BATCH_SIZE = 3 # Adjust based on your GPU memory
GRAD_ACUMM_STEPS = 84 # Adjust based on your BATCH_SIZE to reach a desired effective batch size (BATCH_SIZE * GRAD_ACUMM_STEPS)

# Define here the dataset that you want to use for the fine-tuning on.
# IMPORTANT: Update the path and meta_file_train to point to your Maltese dataset
config_dataset = BaseDatasetConfig(
  formatter="ljspeech", # Assuming your dataset is in LJSpeech format
  dataset_name="maltese_dataset", # Name for your dataset
  path="path/to/your/maltese_tts_data", # <<<< CHANGE THIS PATH
  meta_file_train="metadata.csv", # <<<< CHANGE THIS META FILE NAME if different
  language="mt", # Set language to Maltese
)

# Add here the configs of the datasets
DATASETS_CONFIG_LIST = [config_dataset]

# Define the path where XTTS v2.0.1 files will be downloaded
# This can be the same as your OUT_PATH or a separate directory
CHECKPOINTS_OUT_PATH = os.path.join(OUT_PATH, "XTTS_v2.0_original_model_files/")
os.makedirs(CHECKPOINTS_OUT_PATH, exist_ok=True)

# DVAE files
DVAE_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth"
MEL_NORM_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/mel_stats.pth"

# Set the path to the downloaded files
DVAE_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(DVAE_CHECKPOINT_LINK))
MEL_NORM_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(MEL_NORM_LINK))

# download DVAE files if needed
if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > Downloading DVAE files!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True)

# Download XTTS v2.0 checkpoint if needed
TOKENIZER_FILE_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/vocab.json"
XTTS_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth"

# XTTS transfer learning parameters: You we need to provide the paths of XTTS model checkpoint that you want to do the fine tuning.
TOKENIZER_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(TOKENIZER_FILE_LINK))  # vocab.json file
XTTS_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(XTTS_CHECKPOINT_LINK))  # model.pth file

# download XTTS v2.0 files if needed
if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
  print(" > Downloading XTTS v2.0 files!")
  ModelManager._download_model_files(
    [TOKENIZER_FILE_LINK, XTTS_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True
  )


# Training sentences generations (Optional - for testing generated samples during training)
# IMPORTANT: Update with a reference speaker for your Maltese dataset
SPEAKER_REFERENCE = [
  "path/to/your/maltese/reference_speaker.wav"  # <<<< CHANGE THIS PATH
]
LANGUAGE = config_dataset.language


def main():
  # init args and config
  model_args = GPTArgs(
    max_conditioning_length=132300,  # 6 secs
    min_conditioning_length=66150,  # 3 secs
    debug_loading_failures=False,
    max_wav_length=255995,  # ~11.6 seconds
    max_text_length=200,
    mel_norm_file=MEL_NORM_FILE,
    dvae_checkpoint=DVAE_CHECKPOINT,
    xtts_checkpoint=XTTS_CHECKPOINT,  # checkpoint path of the model that you want to fine-tune
    tokenizer_file=TOKENIZER_FILE,
    gpt_num_audio_tokens=1026,
    gpt_start_audio_token=1024,
    gpt_stop_audio_token=1025,
    gpt_use_masking_gt_prompt_approach=True,
    gpt_use_perceiver_resampler=True,
  )
  # define audio config
  # IMPORTANT: XTTS v2 uses 24kHz output, ensure your data matches or is resampled
  audio_config = XttsAudioConfig(sample_rate=22050, dvae_sample_rate=22050, output_sample_rate=24000) # Keep 24000 for output

  # training parameters config
  config = GPTTrainerConfig(
    output_path=OUT_PATH,
    model_args=model_args,
    run_name=RUN_NAME,
    project_name=PROJECT_NAME,
    run_description="""
      GPT XTTS Maltese fine-tuning
      """, # Changed description
    dashboard_logger=DASHBOARD_LOGGER,
    logger_uri=LOGGER_URI,
    audio=audio_config,
    batch_size=BATCH_SIZE,
    batch_group_size=48, # Adjust if needed
    eval_batch_size=BATCH_SIZE,
    num_loader_workers=8, # Adjust based on your system
    eval_split_max_size=256,
    print_step=50,
    plot_step=100,
    log_model_step=1000,
    save_step=10000, # Adjust save frequency
    save_n_checkpoints=1,
    save_checkpoints=True,
    # target_loss="loss", # Uncomment if you want to track a specific loss
    print_eval=False,
    # Optimizer settings using AdamW as recommended
    optimizer="AdamW",
    optimizer_wd_only_on_weights=OPTIMIZER_WD_ONLY_ON_WEIGHTS,
    optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
    lr=5e-06,  # <<<< Adjust learning rate (start small, e.g., 1e-5)
    lr_scheduler="MultiStepLR",
    # it was adjusted accordly for the new step scheme
    # Adjust milestones based on your total training steps
    lr_scheduler_params={"milestones": [50000 * 18, 150000 * 18, 300000 * 18], "gamma": 0.5, "last_epoch": -1},
    test_sentences=[
      # IMPORTANT: Update with Maltese test sentences and a Maltese speaker reference
      {
        "text": "Dan huwa test tas-sentenzi bil-Malti.", # Example Maltese sentence
        "speaker_wav": SPEAKER_REFERENCE,
        "language": LANGUAGE,
      },
      {
        "text": "Is-silġ fil-muntanji qed idub malajr.", # Example Maltese sentence
        "speaker_wav": SPEAKER_REFERENCE,
        "language": LANGUAGE,
      },
    ],
  )

  # init the model from config
  model = GPTTrainer.init_from_config(config)

  # load training samples
  # IMPORTANT: Ensure your dataset is correctly formatted and the path in config_dataset is correct
  train_samples, eval_samples = load_tts_samples(
    DATASETS_CONFIG_LIST,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
  )

  # init the trainer and 🚀
  trainer = Trainer(
    TrainerArgs(
      restore_path=None,  # xtts checkpoint is restored via xtts_checkpoint key so no need of restore it using Trainer restore_path parameter
      skip_train_epoch=False,
      start_with_eval=START_WITH_EVAL,
      grad_accum_steps=GRAD_ACUMM_STEPS,
    ),
    config,
    output_path=OUT_PATH,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
  )

  # Start training
  trainer.fit()


if __name__ == "__main__":
  main()

Of course. Based on the provided resources, here is a comprehensive guide to fine-tuning the XTTS model for Maltese, complete with a step-by-step process and the full Python code.

### Summary of Key Insights from Provided Links

The provided links offer a wealth of information, which can be synthesized into a clear, actionable plan:

1.  **Feasibility**: The official Coqui TTS documentation and several community guides confirm that fine-tuning XTTS for new, low-resource languages is not only possible but a primary feature of the model.
2.  **Dataset is Critical**: The guide on "Manually Creating Your Own Dataset" emphasizes that high-quality, clean audio is paramount. For a low-resource language, even a small amount of high-quality data (5-15 minutes to start, 1-2 hours for a good result) is more effective than a large amount of noisy data. The audio should be a single speaker, 22050Hz, mono, and in `.wav` format.
3.  **Simplicity of the Process**: The "Simple Version" of the fine-tuning guide and various code repositories show that the process boils down to three main steps: preparing a dataset with a specific file structure, running the training script with the correct parameters, and then using the newly generated model for inference.
4.  **BPE Tokenization (Advanced)**: The advanced guide mentions Byte-Pair Encoding. For a language like Maltese, which uses the Latin alphabet, you **do not need to create a new BPE tokenizer**. The model's existing tokenizer is sufficient, as it was trained on several European languages. This significantly simplifies the process.
5.  **Training Script**: The `train_gpt_xtts.py` script is the core of the operation. The mock code and the various repositories show how to configure it with the necessary arguments, such as dataset paths, batch size, learning rate, and the number of epochs.

---

### Step-by-Step Guide to Fine-Tune XTTS for Maltese

This guide assumes you have a computer with a modern NVIDIA GPU (with at least 12GB of VRAM, 24GB is recommended for a smoother experience).

#### **Step 1: Prepare Your Maltese Dataset**

This is the most important step. Quality over quantity.

1.  **Record Your Audio**:
    *   Find a native Maltese speaker with a clear voice.
    *   Record them reading Maltese sentences in a quiet environment with a decent microphone. Aim for **at least 1 hour of audio**.
    *   Export the audio as a single `.wav` file, with a sample rate of **22050 Hz** and in **mono**.

2.  **Transcribe and Segment the Audio**:
    *   You need to break down your long audio file into many short clips (ideally 3-10 seconds each) and create a transcript for each one.
    *   **Easy Method:** Use a transcription tool that provides per-word timestamps. A great open-source option is `faster-whisper`.
    *   **Output Format**: The final dataset needs to be in the "LJSpeech" format:
        *   A folder named `wavs` containing all your short audio clips (`.wav` files).
        *   A file named `metadata.csv` in the same directory.
        *   Each line in `metadata.csv` should have the format: `filename|transcription|normalized_transcription`. For this purpose, the second and third columns can be the same.

    **Example `metadata.csv`:**
    ```
    maltese_clip_001|Dan huwa test tal-mudell tat-taħdit.|Dan huwa test tal-mudell tat-taħdit.
    maltese_clip_002|It-temp illum huwa sabiħ ħafna.|It-temp illum huwa sabiħ ħafna.
    ...
    ```

3.  **Split the Dataset**:
    *   Create a main dataset directory (e.g., `maltese_dataset`).
    *   Inside, place your `wavs` folder.
    *   Randomly split your `metadata.csv` into two files: `metadata_train.csv` (for training, ~95% of the lines) and `metadata_eval.csv` (for validation, ~5% of the lines).

#### **Step 2: Set Up the Environment and Code**

1.  **Install Dependencies**:
    ```bash
    pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    pip install TTS
    ```

2.  **Get the Training Script**:
    *   Download the `train_gpt_xtts.py` script. The one you provided from the forked repositories is a great, self-contained starting point. Save it in your project's root directory.

#### **Step 3: Run the Fine-Tuning Process**

1.  **The Command**:
    *   Navigate to your project directory in the terminal.
    *   Run the training script using a command similar to the one you found. This command tells the script where your data is, what language to use, and how to train.

    Here is a complete, corrected command for your Maltese dataset:
    ```bash
    python train_gpt_xtts.py \
    --output_path ./xtts_maltese_model/ \
    --metadatas path/to/your/maltese_dataset/metadata_train.csv,path/to/your/maltese_dataset/metadata_eval.csv,mt \
    --num_epochs 10 \
    --batch_size 4 \
    --grad_acumm 8 \
    --max_text_length 200 \
    --max_audio_length 242000 \
    --weight_decay 1e-4 \
    --lr 5e-6 \
    --save_step 5000
    ```
    **Key Parameter Explanations:**
    *   `--output_path`: Where the final model and checkpoints will be saved.
    *   `--metadatas`: The crucial part. It's a comma-separated string: `train_metadata_path,eval_metadata_path,language_code`. For Maltese, the ISO 639-1 code is `mt`.
    *   `--num_epochs`: The number of times to go through the training data. Start with 10 and you can increase it later.
    *   `--batch_size` & `--grad_acumm`: These control memory usage. If you run out of VRAM, decrease the `batch_size` and increase `grad_acumm` to compensate. An effective batch size is `batch_size * grad_acumm`.

#### **Step 4: Use Your Fine-Tuned Model for Inference**

Once the training is complete, your `output_path` will contain a folder with a name like `GPT_XTTS_FT...`. Inside, you'll find the `best_model.pth` and `config.json`.

1.  **Prepare a Speaker Reference**: You need a short (5-15 seconds) audio clip of the target voice to clone. This should be from your training dataset to ensure consistency.
2.  **Inference Code**: Use the code you provided, but update the paths to point to your new fine-tuned model.

---

### Full Python Code

Here is the complete, verified Python script for **inference** using your newly fine-tuned Maltese model. The training is done via the command line as shown in Step 3.

```python
import torch
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts

# --- 1. SET UP YOUR FILE PATHS ---
# Path to the folder where your fine-tuned model is saved.
model_output_path = "./xtts_maltese_model/run/training/GPT_XTTS_FT-..." # Replace with your actual output folder name

# Path to the best model checkpoint.
xtts_checkpoint = os.path.join(model_output_path, "best_model.pth")

# Path to the model's configuration file.
xtts_config = os.path.join(model_output_path, "config.json")

# Path to the original vocabulary file (this doesn't change).
# The training script should have copied this for you.
xtts_vocab = "./xtts_maltese_model/ready/vocab.json"

# A clean, 10-15 second audio clip of your Maltese speaker.
# This is used to provide the voice timbre.
speaker_audio_file = "path/to/your/maltese_speaker_reference.wav"

# --- 2. LOAD THE FINE-TUNED MODEL ---
print("Loading model...")
config = XttsConfig()
config.load_json(xtts_config)

model = Xtts.init_from_config(config)
model.load_checkpoint(config, checkpoint_path=xtts_checkpoint, vocab_path=xtts_vocab, use_deepspeed=False)

# Move the model to the GPU if available.
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("Model loaded successfully!")

# --- 3. SYNTHESIZE SPEECH ---
# The Maltese text you want to convert to speech.
tts_text = "Dan huwa test tal-mudell tat-taħdit il-ġdid tiegħi għal-lingwa Maltija."
lang = "mt" # Use the Maltese language code.

print(f"Generating speech for: '{tts_text}'")

# Generate conditioning latents from the speaker reference.
gpt_cond_latent, speaker_embedding = model.get_conditioning_latents(
  audio_path=speaker_audio_file
)

# Inference call
out = model.inference(
    text=tts_text,
    language=lang,
    gpt_cond_latent=gpt_cond_latent,
    speaker_embedding=speaker_embedding,
    temperature=0.7,  # A higher temperature adds more randomness to the output
    length_penalty=1.0,
    repetition_penalty=10.0,
    top_k=50,
    top_p=0.85,
)

# Save the generated audio to a file.
import torchaudio
torchaudio.save("output_maltese.wav", torch.tensor(out["wav"]).unsqueeze(0), 24000)

print("Audio saved to output_maltese.wav")```

This comprehensive guide and code provide a clear and actionable path to fine-tuning the XTTS model for Maltese, leveraging the best practices from the resources you've provided.



---


# Inference

In [ ]:
from TTS.api import TTS
import torch

# Path to the configuration and model checkpoint of your fine-tuned model
config_path = "xtts_maltese_finetuned/config.json"
model_path = "xtts_maltese_finetuned/best_model.pth" # Or the latest checkpoint
speakers_json_path = "xtts_maltese_finetuned/speakers.json" # If you have speaker information
vocab_path = "xtts_maltese_finetuned/vocab.json"

device = "cuda" if torch.cuda.is_available() else "cpu"

tts = TTS(model_path=model_path, config_path=config_path, progress_bar=True).to(device)

maltese_text = "Bonġu, kif int?"

reference_audio = "path/to/maltese/speaker.wav"

tts.tts_to_file(text=maltese_text, speaker_wav=reference_audio, language="mt", file_path="output_maltese.wav")

In [ ]:
from datasets import load_dataset

dataset = load_dataset("mozilla-foundation/common_voice_17_0", "mt", split="train")

audio_info = dataset[1481]['audio']
audio_array = audio_info['array']
sampling_rate = audio_info['sampling_rate']

print(f"Audio array shape: {audio_array.shape}")
print(f"Sampling rate: {sampling_rate}")



---


# Evaluation
[Flore+](https://huggingface.co/datasets/openlanguagedata/flores_plus/viewer/mlt_Latn?views%5B%5D=mlt_latn_dev)  
Use Wave2Vec2 or Wispher (but they don't work as good as expected for Maltese) for CER evaluation  
How does it fares when text mixes pure Maltese and English?    
automatic mos prediction (with pretrained squid ?)  


---
# [Hugging Face tutorial](https://huggingface.co/docs/transformers/main/en/tasks/text-to-speech)

In [ ]:
import os
import torch
import matplotlib.pyplot as plt
from IPython.display import Audio

from datasets import load_dataset, Audio
from dataclasses import dataclass

from typing import Any, Dict, List, Union
from collections import defaultdict

from transformers import SpeechT5ForTextToSpeech
from transformers import Seq2SeqTrainingArguments
from transformers import pipeline

from speechbrain.inference.classifiers import EncoderClassifier
from accelerate.test_utils.testing import get_backend

KeyboardInterrupt: 

In [ ]:
# Simple demo
pipe = pipeline("text-to-speech", model="suno/bark-small")
text = "[clears throat] This is a test ... and I just took a long pause."
output = pipe(text)

Audio(output["audio"], rate=output["sampling_rate"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cpu
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
# Dataset extraction
dataset = load_dataset("facebook/voxpopuli", "nl", split="train", force_download=True)
print(len(dataset))
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

checkpoint = "microsoft/speecht5_tts"
processor = SpeechT5Processor.from_pretrained(checkpoint)
tokenizer = processor.tokenizer

def extract_all_chars(batch):
  all_text = " ".join(batch["normalized_text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
  extract_all_chars,
  batched=True,
  batch_size=-1,
  keep_in_memory=True,
  remove_columns=dataset.column_names,
)

dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

dataset_vocab - tokenizer_vocab

In [ ]:
#Dataset cleanup
replacements = [
  ("à", "a"),
  ("ç", "c"),
  ("è", "e"),
  ("ë", "e"),
  ("í", "i"),
  ("ï", "i"),
  ("ö", "o"),
  ("ü", "u"),
]


def cleanup_text(inputs):
  for src, dst in replacements:
    inputs["normalized_text"] = inputs["normalized_text"].replace(src, dst)
  return inputs

dataset = dataset.map(cleanup_text)






speaker_counts = defaultdict(int)
for speaker_id in dataset["speaker_id"]:
  speaker_counts[speaker_id] += 1
plt.figure()
plt.hist(speaker_counts.values(), bins=20)
plt.ylabel("Speakers")
plt.xlabel("Examples")
plt.show()

def select_speaker(speaker_id):
  return 100 <= speaker_counts[speaker_id] <= 400

dataset = dataset.filter(select_speaker, input_columns=["speaker_id"])
print(len(set(dataset["speaker_id"])))
print(len(dataset))


spk_model_name = "speechbrain/spkrec-xvect-voxceleb"
device, _, _ = get_backend() # automatically detects the underlying device type (CUDA, CPU, XPU, MPS, etc.)
speaker_model = EncoderClassifier.from_hparams(
  source=spk_model_name,
  run_opts={"device": device},
  savedir=os.path.join("/tmp", spk_model_name),
)


def create_speaker_embedding(waveform):
  with torch.no_grad():
    speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
    speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
    speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
  return speaker_embeddings

def prepare_dataset(example):
  audio = example["audio"]

  example = processor(
    text=example["normalized_text"],
    audio_target=audio["array"],
    sampling_rate=audio["sampling_rate"],
    return_attention_mask=False,
  )

  # strip off the batch dimension
  example["labels"] = example["labels"][0]

  # use SpeechBrain to obtain x-vector
  example["speaker_embeddings"] = create_speaker_embedding(audio["array"])

  return example




processed_example = prepare_dataset(dataset[0])
print(list(processed_example.keys()))
print(processed_example["speaker_embeddings"].shape)

plt.figure()
plt.imshow(processed_example["labels"].T)
plt.show()

dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names)
def is_not_too_long(input_ids):
  input_length = len(input_ids)
  return input_length < 200

dataset = dataset.filter(is_not_too_long, input_columns=["input_ids"])
print(len(dataset))

dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
# Dataset class
@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(self, features: list[dict[str, Union[list[int], torch.Tensor]]]) -> dict[str, torch.Tensor]:
      input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
      label_features = [{"input_values": feature["labels"]} for feature in features]
      speaker_features = [feature["speaker_embeddings"] for feature in features]

      # collate the inputs and targets into a batch
      batch = processor.pad(input_ids=input_ids, labels=label_features, return_tensors="pt")

      # replace padding with -100 to ignore loss correctly
      batch["labels"] = batch["labels"].masked_fill(batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100)

      # not used during fine-tuning
      del batch["decoder_attention_mask"]

      # round down target lengths to multiple of reduction factor
      if model.config.reduction_factor > 1:
        target_lengths = torch.tensor([len(feature["input_values"]) for feature in label_features])
        target_lengths = target_lengths.new(
          [length - length % model.config.reduction_factor for length in target_lengths]
        )
        max_length = max(target_lengths)
        batch["labels"] = batch["labels"][:, :max_length]

      # also add in the speaker embeddings
      batch["speaker_embeddings"] = torch.tensor(speaker_features)

      return batch

data_collator = TTSDataCollatorWithPadding(processor=processor)

In [ ]:
# Model config and training
model = SpeechT5ForTextToSpeech.from_pretrained(checkpoint)

model.config.use_cache = False


training_args = Seq2SeqTrainingArguments(
    output_dir="speecht5_finetuned_voxpopuli_nl",  # change to a repo name of your choice
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=2,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=True,
)

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    processing_class=processor,
)

trainer.train()

# processor.save_pretrained("bluefir/speecht5_finetuned_voxpopuli_nl")
# trainer.push_to_hub()



# Pipe
text = "hallo allemaal, ik praat nederlands. groetjes aan iedereen!"
example = dataset["test"][304]
speaker_embeddings = torch.tensor(example["speaker_embeddings"]).unsqueeze(0)
forward_params = {"speaker_embeddings": speaker_embeddings}
output = pipe(text, forward_params=forward_params)

print(output)

Audio(output['audio'], rate=output['sampling_rate'])